 Import Libraries

In [4]:
import os
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image


Define the Dataset Class

In [5]:
class ElephantDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = []
        
        for label, subfolder in enumerate(['elephant', 'not_elephant']):
            folder_path = os.path.join(root_dir, subfolder)
            self.image_files.extend([(os.path.join(folder_path, f), label) 
                                     for f in os.listdir(folder_path) 
                                     if f.endswith(('.png', '.jpg', '.jpeg'))])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path, label = self.image_files[idx]
        image = Image.open(img_path).convert("RGB")
        
        if self.transform:
            image = self.transform(image)
            
        return image, label


Set Up Data Transformations and Data Loaders

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = ElephantDataset(root_dir='data/train', transform=transform)
val_dataset = ElephantDataset(root_dir='data/val', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=False, num_workers=0)


Define and Set Up the Model

In [5]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 1)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


C:\Users\sweet\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\sweet\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Define the Training Loop

In [6]:
def train_model(model, criterion, optimizer, train_loader, val_loader, epochs=20):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device, dtype=torch.float32)
            
            optimizer.zero_grad()
            outputs = model(images).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {train_loss/len(train_loader):.4f}")
        
        # Validation
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device, dtype=torch.float32)
                outputs = model(images).squeeze()
                predicted = torch.round(torch.sigmoid(outputs))
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        print(f"Validation Accuracy: {100 * correct / total:.2f}%")


Train the Model

In [7]:
train_model(model, criterion, optimizer, train_loader, val_loader, epochs=10)


Epoch [1/10], Loss: 0.2350
Validation Accuracy: 93.02%
Epoch [2/10], Loss: 0.1023
Validation Accuracy: 93.93%
Epoch [3/10], Loss: 0.0642
Validation Accuracy: 91.95%
Epoch [4/10], Loss: 0.0670
Validation Accuracy: 96.96%
Epoch [5/10], Loss: 0.0406
Validation Accuracy: 95.80%
Epoch [6/10], Loss: 0.0475
Validation Accuracy: 90.84%
Epoch [7/10], Loss: 0.0487
Validation Accuracy: 97.42%
Epoch [8/10], Loss: 0.0267
Validation Accuracy: 96.66%
Epoch [9/10], Loss: 0.0164
Validation Accuracy: 98.13%
Epoch [10/10], Loss: 0.0071
Validation Accuracy: 97.22%


saving the model

In [14]:
# Save the model
torch.save(model.state_dict(),"elephant_image_classifier.ph")
print("Model saved successfully.")

Model saved successfully.


defining predict class

In [6]:
def predict(image_path, model):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.eval()
    model.to(device)
    
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        output = model(image).squeeze()
        prediction = torch.sigmoid(output).item()
        return "Not Elephant" if prediction >= 0.8 else "Elephant"
    



In [14]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 1)

C:\Users\sweet\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\sweet\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


load the model

In [15]:
model.load_state_dict(torch.load("./elephant_image_classifier.ph"))

C:\Users\sweet\AppData\Local\Temp\ipykernel_15764\599037389.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("./elephant_image_classifier

<All keys matched successfully>

evaluating the model

In [16]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

testing

In [17]:

test_image_path1 = 'data/test/Elephant_968.jpg'
test_image_path2 = 'data/test/elephant.jpg'
test_image_path3 = 'data/test/Zebra_99.jpg'
test_image_path4 = 'data/test/Rhino_98.jpg'
print(predict(test_image_path1, model))
print(predict(test_image_path2, model))
print(predict(test_image_path3, model))
print(predict(test_image_path4, model))

Elephant
Elephant
Not Elephant
Not Elephant
